<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/agent/openai_agent_with_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NVIDIA Agent with Query Engine Tools

## Build Query Engine Tools

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install --upgrade --quiet llama-index-agent-nvidia

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install --quiet llama-index

In [ ]:
import getpass
import os

# del os.environ['NVIDIA_API_KEY']  ## delete key and reset
if os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    print("Valid NVIDIA_API_KEY already in environment. Delete to reset")
else:
    nvapi_key = getpass.getpass("NVAPI Key (starts with nvapi-): ")
    assert nvapi_key.startswith(
        "nvapi-"
    ), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

from llama_index.llms.nvidia import NVIDIA
from llama_index.embeddings.nvidia import NVIDIAEmbedding

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [ ]:
# Use ollama in JSON mode
from llama_index.core import Settings

Settings.llm = NVIDIA(
    "meta/llama-3.1-70b-instruct", is_function_calling_model=True
)
Settings.embed_model = NVIDIAEmbedding()

In [ ]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

Download Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [ ]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Setup NVIDIAAgent Agent

In [ ]:
from llama_index.agent.nvidia import NVIDIAAgent

In [ ]:
agent = NVIDIAAgent.from_tools(query_engine_tools, verbose=True)

## Let's Try It Out!

In [ ]:
agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Added user message to memory: Is network of Light Vehicles affected for lyft


=== Calling Function ===
Calling function: lyft_10k with args: {"input": "Is network of Light Vehicles affected for lyft?"}


Assistant: Lyft's network of light vehicles was not significantly affected in 2021.

Added user message to memory: I meant due to covid-19


=== Calling Function ===
Calling function: lyft_10k with args: {"input": "lyft network of light vehicles affected due to covid-19?"}


Assistant: Lyft's network of light vehicles was affected due to the COVID-19 pandemic.

Added user message to memory: can you eloborate on how?


=== Calling Function ===
Calling function: lyft_10k with args: {"input": "How lyft network of light vehicles affected due to covid-19?"}


Assistant: The COVID-19 pandemic had a significant impact on Lyft's network of light vehicles. Due to the pandemic, there was a decrease in demand for ride-hailing services, which resulted in a reduction in the number of active drivers on the platform. This, in turn, affected the availability of light vehicles for riders. Additionally, the pandemic led to an increase in costs for Lyft, including costs related to safety measures and support for drivers.

